<h2>Intrinsic Matrix

In [ ]:
import cv2 as cv
import numpy as np
from glob import glob
from matplotlib import pyplot as plt
import PIL.ExifTags
from bisect import bisect_left

<h2>Constants

In [ ]:
MIN_NUM_MATCHES = 100

<h2>UTILS

In [ ]:
def show_images(images,titles=None):
    #This function is used to show image(s) with titles by sending an array of images and an array of associated titles.
    # images[0] will be drawn with the title titles[0] if exists
    # You aren't required to understand this function, use it as-is.
    n_ims = len(images)
    if titles is None: titles = ['(%d)' % i for i in range(1,n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image,title in zip(images,titles):
        a = fig.add_subplot(1,n_ims,n)
        if image.ndim == 2: 
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show()

<h2>Read Images

In [ ]:
datasetPath = "Data/dinoSparseRing/"
images = [cv.imread(file) for file in glob(datasetPath+'*.png')]

<h2>Read EXIF Info

In [ ]:
img = PIL.Image.open('tryexif.jpg')
exif = {
    PIL.ExifTags.TAGS[k]: v
    for k, v in img._getexif().items()
    if k in PIL.ExifTags.TAGS
}

<h2>SIFT

In [ ]:
imagesModels = []
for img in images:
    sift = cv.xfeatures2d.SIFT_create()
    # find keypoints and descriptors with SIFT
    kp,des = sift.detectAndCompute(img, None)
    imageModel = {
        "keyPts": kp,
        "descriptors": des
    }
    imagesModels.append(imageModel)

<h2> Matching

In [ ]:
# Get the fundmental matrix between 2 pictures
matchesIdx = [[list() for x in range(len(images))] for y in range(len(images))] 
def compute_matches(idx1,idx2):
    #Matching descriptors
    # FLANN parameters
    FLANN_INDEX_KDTREE = 0 #TODO try:1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)

    flann = cv.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(imagesModels[idx1]["descriptors"], imagesModels[idx2]["descriptors"], k=2)
    
    pts1 = []
    pts2 = []
    #Need to draw only good matches, so create a mask
    matchesMask = [[0,0] for i in range(len(matches))]
    for i,(m,n) in enumerate(matches):
        if m.distance < 0.6*n.distance:
            #pt = list(kp2[m.trainIdx].pt)
            #pt.append(1) #t
            pts2.append(imagesModels[idx2]["keyPts"][m.trainIdx].pt)
            #pt = list(kp1[m.queryIdx].pt)
            #pt.append(1) #t
            pts1.append(imagesModels[idx1]["keyPts"][m.queryIdx].pt) 
            
            matchesIdx[idx1][idx2].append({'idx1': m.queryIdx, 'idx2':  m.trainIdx})
            matchesIdx[idx2][idx1].append({'idx1': m.trainIdx, 'idx2': m.queryIdx})
            matchesMask[i] = [1,0]
    
    #Draw Matches
    draw_params = dict(matchColor = (0,255,0),
                   singlePointColor = (255,0,0),
                   matchesMask = matchesMask,
                   flags = cv.DrawMatchesFlags_DEFAULT)
    img3 = cv.drawMatchesKnn(images[idx1], imagesModels[idx1]["keyPts"], images[idx2], imagesModels[idx2]["keyPts"], matches,None,**draw_params)
    show_images([images[idx1], images[idx2], img3], ["img1: idx " + str(idx1), "img2: idx "+str(idx2), "matches"])
    
    return pts1, pts2
    '''
    pts1 = np.float32(pts1)
    pts2 = np.float32(pts2)
    #print(pts1)
    #print(pts2)
    ;;;;;;;;;;;;;;;;;;;;;;;
    print("pts1.shape:%s\tpts2.shape:%s"%(pts1.shape,pts2.shape))
    fundmentalMat, mask = cv.findFundamentalMat(pts1,pts2,cv.FM_LMEDS)
    # We select only inlier points
    #pts1 = pts1[mask.ravel()==1]
    #pts2 = pts2[mask.ravel()==1]
    print(("Fundmental Matrix between image[%d] and image[%d]:\n%a") % (idx1,idx2,fundmentalMat))
    return fundmentalMat
    '''

<h2>Intrinsic Matrix

In [ ]:
k = np.zeros((3,3))
focalLen = 0.5
k = [[focalLen, 0, 0],
     [0, focalLen, 0],
     [0, 0, 1]]
print(k)

<h2>Compute Fundamental Matrix

In [ ]:
def compute_fundamental(x1,x2):
    """ Computes the fundamental matrix from corresponding points
    (x1,x2 3*n arrays) using the normalized 8 point algorithm.
    each row is constructed as
    [x’*x, x’*y, x’, y’*x, y’*y, y’, x, y, 1] """
    n = x1.shape[1]
    if x2.shape[1] != n:
        raise ValueError("Number of points don’t match.")
    # build matrix for equations
    A = np.zeros((n,9))
    for i in range(n):
        A[i] = [x1[0,i]*x2[0,i], x1[0,i]*x2[1,i], x1[0,i]*x2[2,i],
        x1[1,i]*x2[0,i], x1[1,i]*x2[1,i], x1[1,i]*x2[2,i],
        x1[2,i]*x2[0,i], x1[2,i]*x2[1,i], x1[2,i]*x2[2,i] ]
    # compute linear least square solution
    U,S,V = np.linalg.svd(A)
    F = V[-1].reshape(3,3)
    # constrain F
    # make rank 2 by zeroing out last singular value
    U,S,V = np. linalg.svd(F)
    S[2] = 0
    F = np.dot(U,np.dot(np.diag(S),V))
    return F

In [ ]:
def compute_fundamental_normalized(x1,x2):
    """ Computes the fundamental matrix from corresponding points
    (x1,x2 3*n arrays) using the normalized 8 point algorithm. """
    n = x1.shape[1]
    if x2.shape[1] != n:
        raise ValueError("Number of points don’t match.")
    # normalize image coordinates
    x1 = x1 / x1[2]
    mean_1 = np.mean(x1[:2],axis=1)
    S1 = np.sqrt(2) / np.std(x1[:2])
    T1 = np.array([[S1,0,-S1*mean_1[0]],[0,S1,-S1*mean_1[1]],[0,0,1]])
    x1 = np.dot(T1,x1)
    x2 = x2 / x2[2]
    mean_2 = np.mean(x2[:2],axis=1)
    S2 = np.sqrt(2) / np.std(x2[:2])
    T2 = np.array([[S2,0,-S2*mean_2[0]],[0,S2,-S2*mean_2[1]],[0,0,1]])
    x2 = np.dot(T2,x2) 
    # compute F with the normalized coordinates
    F = compute_fundamental(x1,x2)
    # reverse normalization
    F = np.dot(np.transpose(T1),np.dot(F,T2))
    return F/F[2,2]

In [ ]:
class RansacModel(object):
    """ Class for fundmental matrix fit with ransac.py from
    http://www.scipy.org/Cookbook/RANSAC"""
    def __init__(self,debug=False):
        self.debug = debug
        
    def fit(self,data):
        """ Estimate fundamental matrix using eight
        selected correspondences. """
        # transpose and split data into the two point sets
        data = np.transpose(data)
        x1 = data[:3,:8]
        x2 = data[3:,:8]
        # estimate fundamental matrix and return
        F = compute_fundamental_normalized(x1,x2)
        return F
    
    def get_error(self,data,F):
        """ Compute x^T F x for all correspondences,
        return error for each transformed point. """
        # transpose and split data into the two point
        data = np.transpose(data)
        x1 = data[:3]
        x2 = data[3:]
        # Sampson distance as error measure
        Fx1 = np.dot(F,x1)
        Fx2 = np.dot(F,x2)
        denom = Fx1[0]**2 + Fx1[1]**2 + Fx2[0]**2 + Fx2[1]**2
        err = ( np.diag(np.dot(np.transpose(x1),np.dot(F,x2))) )**2 / denom
        # return error per point
        return err
        

In [ ]:
def F_from_ransac(x1,x2,model,maxiter=5000,match_theshold=1e-6):
    """ Robust estimation of a fundamental matrix F from point
    correspondences using RANSAC (ransac.py from
    http://www.scipy.org/Cookbook/RANSAC).
    input: x1,x2 (3*n arrays) points in hom. coordinates. """
    import ransactmp as ransac
    data = np.vstack((x1,x2))
    # compute F and return with inlier index
    F,ransac_data = ransac.ransac(np.transpose(data),model,8,maxiter,match_theshold,20,debug = False,return_all=True)
    return F, ransac_data['inliers']

<h2>Main

In [ ]:
#matchScores = np.zeros((len(images), len(images)))
homographyScores = np.zeros((len(images), len(images)))
#matches = np.zeros((len(images), len(images), 2, 2)) #TODO "2 or 3 if t is added"
# estimate E with RANSAC
model = RansacModel()
for i in range(len(images)):
    for j in range(i+1, len(images)):
        pts1, pts2 = compute_matches(i,j)
        #matchScores[i][j] = len(pts1)
        #matchScores[j][i] = len(pts1)
        print("No. of features between ", str(i) , "&", str(j),len(pts1))

        '''
        #TODO Add "t" to every point
        pts1 = np.transpose(pts1)
        pts2 = np.transpose(pts2)
        if(len(pts1) < 8):
            print("Couldn't find good 8 matches to compute Fundamental matrix")
        else:
            F,inliers = F_from_ransac(pts1,pts2,model,maxiter=2048,match_theshold=1)
            print("Fundamental Matrix between ", str(i) , "&", str(j), F)
        '''
       #matches[i][j] = [pts1, pts2]
       #matches[j][i] = [pts2, pts1]
        
        pts1 = np.float32(pts1).reshape(-1,1,2)
        pts2 = np.float32(pts2).reshape(-1,1,2)
        h, w, _ = images[i].shape
        #H, _ = cv.findHomography(pts1, pts2, cv.RANSAC, 0.004*max(h,w))
        #if len(H) ==  0:
        #    print("H matrix cannot be estimated")
        #else:
        #   if matchScores[i][j] >= MIN_NUM_MATCHES:
        #        homographyScores[i][j] = 
#sort homographyScores
#choose min

<h2> Tracks

In [ ]:
class KeyList(object):
    # bisect doesn't accept a key function, so we build the key into our sequence.
    def __init__(self, l, key):
        self.l = l
        self.key = key
    def __len__(self):
        return len(self.l)
    def __getitem__(self, index):
        return self.key(self.l[index])
    

def get_matched_idx(matchesList, searchItem):
    i = bisect_left(KeyList(matchesList, key=lambda k: k['idx1']), searchItem) 
    if i != len(matchesList) and matchesList[i]['idx1'] == searchItem: return matchesList[i]['idx2']
    else: return -1
    
#Test 
'''
matchesList = [{'idx1': 100, 'idx2': 5}, {'idx1': 2, 'idx2': 88}, {'idx1': 7, 'idx2': 5}, {'idx1': 9, 'idx2': 10}]
print(matchesList[0]['idx1'])
matchesList = sorted(matchesList, key=lambda k: k["idx1"]) 
print(matchesList[0]['idx1'])
print(get_matched_idx(matchesList, 2))
'''

In [ ]:
tracks = []
#Sort matchesIdx #TODO #find faster way 
for i in range(len(images)):
    for j in range(len(images)):
        matchesIdx[i][j] =  sorted(matchesIdx[i][j], key=lambda k: k["idx1"])

for imgIdx in range(len(images)):
    if len(matchesIdx[imgIdx]) == 0: continue #??TODO neighbors
    flags = np.zeros(len(imagesModels[imgIdx]['keyPts']))
    imagesModels[imgIdx]['keyFlags'] = flags
    
for imgIdx in range(len(images)):
    if len(matchesIdx[imgIdx]) == 0: continue #??TODO neighbors
    '''
    BFS from each feature
    images --> Nodes
    Features --> Egdes
    '''
    for featureIdx in range(len(imagesModels[imgIdx]['keyPts'])):
        #if the feature was visited
        if imagesModels[imgIdx]['keyFlags'][featureIdx] == 1: continue
        featuresTrack = []
        featuresQueue = []
        imageVisited = np.zeros(len(images))
        #Mark the feature as visited
        imagesModels[imgIdx]['keyFlags'][featureIdx] = 1
        featuresTrack.append((imgIdx, featureIdx))
        featuresQueue.append((imgIdx, featureIdx))
        imageVisited[imgIdx] = 1
        while not len(featuresQueue) == 0:
            feature = featuresQueue.pop(0)
            parImgIdx = feature[0]
            parFeatureIdx = feature[1]
            for childIdx in range(len(matchesIdx[parImgIdx])): #??TODO neighbors
                if imageVisited[childIdx] == 1: continue
                childFeatureIdx = get_matched_idx(matchesIdx[parImgIdx][childIdx], parFeatureIdx)# search for parFeatureIdx get the corresponding idx
                if childFeatureIdx == -1: continue
                if imagesModels[childIdx]['keyFlags'][childFeatureIdx] == 1: continue
                imagesModels[childIdx]['keyFlags'][childFeatureIdx] = 1
                featuresTrack.append((childIdx, childFeatureIdx))
                featuresQueue.append((childIdx, childFeatureIdx))
                imageVisited[childIdx] = 1
        if len(featuresTrack) >= 2:
            tracks.append(featuresTrack)